# Migration between Scotland and Overseas

In [125]:
from gssutils import *
scraper = Scraper('https://www.nrscotland.gov.uk/statistics-and-data/statistics/statistics-by-theme/' \
                  'migration/migration-statistics/migration-flows/migration-between-scotland-and-overseas')
scraper

## Migration between Scotland and Overseas

### Description

Migration between Scotland and overseas refers to people moving between
Scotland and any country outside the UK.

Due to the sources of data used by National Records of Scotland (NRS) to
estimate migration, the country or group of countries that overseas migrants
come from cannot be identified.



### Distributions

1. Migration between administrative areas and overseas by sex ([MS Excel Spreadsheet](https://www.nrscotland.gov.uk/files//statistics/migration/flows/apr-19/mig-overseas-admin-sex-tab1.xlsx))
1. Migration between administrative areas and overseas by sex ([text/csv](https://www.nrscotland.gov.uk/files//statistics/migration/flows/apr-19/mig-overseas-admin-sex-tab1.zip))
1. Migration between Scotland and overseas by age ([MS Excel Spreadsheet](https://www.nrscotland.gov.uk/files//statistics/migration/flows/apr-19/mig-overseas-syoa-tab2.xlsx))
1. Migration between Scotland and overseas by age ([text/csv](https://www.nrscotland.gov.uk/files//statistics/migration/flows/apr-19/mig-overseas-syoa-tab2.zip))


In [126]:
scraper.dataset.theme = metadata.THEME['population']
scraper.dataset

In [127]:
databaker_sheets = {sheet.name: sheet for sheet in scraper.distribution(
    title='Migration between administrative areas and overseas by sex',
    mediaType=Excel).as_databaker()}

In [128]:
next_table = pd.DataFrame()

In [129]:
%%capture

tab = databaker_sheets['Net-Council Area-Sex']
%run "migration-admin-areas-by-sex-net.ipynb"
next_table = pd.concat([next_table, tidy])

tab = databaker_sheets['In-Council Area-Sex']
%run "migration-admin-areas-by-sex-in.ipynb"
next_table = pd.concat([next_table, tidy])

tab = databaker_sheets['Out-Council Area-Sex']
%run "migration-admin-areas-by-sex-out.ipynb"
next_table = pd.concat([next_table, tidy])



In [130]:
 distribution = scraper.distribution(
    title='Migration between Scotland and overseas by age',
    mediaType='application/vnd.ms-excel')
tabs = distribution.as_databaker()

In [131]:
%run "migration-by-age-2001-to-2017.ipynb"
next_table = pd.concat([next_table, Final_table])

In [132]:
tab = distribution.as_pandas(sheet_name = 'SYOA Females (2001-)')
%run "migration-by-age-2001-to-2017-females.ipynb"
next_table = pd.concat([next_table, Final_table])

In [133]:
%run "migration-by-age-2001-to-2017-persons.ipynb"
next_table = pd.concat([next_table, Final_table])

In [134]:
%run "migration-by-age-2001-to-2017-males.ipynb"
next_table = pd.concat([next_table, Final_table])

In [135]:
next_table.columns = ['Domestic geography1' if x=='Domestic geography' else x for x in next_table.columns]

In [136]:
import pandas as pd
c=pd.read_csv("scottish-geo-lookup.csv")

In [137]:
c

,label,notation
0,Scotland,S92000003
1,Clackmannanshire,S12000005
2,Glasgow City,S12000046
3,Dumfries and Galloway,S12000006
4,East Ayrshire,S12000008
5,East Lothian,S12000010
6,East Renfrewshire,S12000011
7,Falkirk,S12000014
8,Fife,S12000015
9,Highland,S12000017


In [138]:
table = pd.merge(next_table, c, how = 'left', left_on = 'Domestic geography1', right_on = 'label')

In [139]:
table.columns = ['Domestic geography' if x=='notation' else x for x in table.columns]

In [140]:
table['Domestic geography'].fillna('None', inplace = True)

In [141]:
def user_perc(x,y):
    
    if x == 'None' :
        return y
    else:
        return x
    
table['Domestic geography'] = table.apply(lambda row: user_perc(row['Domestic geography'], row['Domestic geography1']), axis = 1)


In [142]:
table = table[['Domestic geography','Foreign geography','Mid Year','Sex','Age', 'Flow','Measure Type','Value','Unit']]

In [143]:
table['Value'] = table['Value'].astype(int)

In [144]:
table = table[table['Mid Year'] != '']

In [145]:
table = table[table['Mid Year'] != 'Year']

In [146]:
table['Age'] = table['Age'].map(
    lambda x: {
        'nrs/all' : 'all', 
        'year/all' : 'all',
        }.get(x, x))


In [147]:
table['Flow'] = table['Flow'].str.lower()

In [148]:
table['Flow'] = table['Flow'].map(
    lambda x: {
        'total' : 'resident'
        }.get(x, x))

In [149]:
table = table[table['Flow']  != 'resident']

In [150]:
from pathlib import Path
out = Path('out')
out.mkdir(exist_ok=True)
table.drop_duplicates().to_csv(out / 'tidy.csv', index = False)

In [151]:
from gssutils.metadata import THEME

scraper.dataset.family = 'migration'
scraper.dataset.theme = THEME['population']
scraper.dataset.license = 'http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/'

with open(out / 'dataset.trig', 'wb') as metadata:
    metadata.write(scraper.generate_trig())